In [0]:
#file - location : /FileStore/tables/776803233_T_T100D_SEGMENT_US_CARRIER_ONLY.csv
data = spark.read.option("header", "true").csv("/FileStore/tables/776803233_T_T100D_SEGMENT_US_CARRIER_ONLY.csv").select("ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID")
data.show(5)

+-----------------+---------------+
ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|
+-----------------+---------------+
 10006| 10056|
 10006| 10056|
 10011| 11869|
 10011| 10661|
 10011| 10661|
+-----------------+---------------+
only showing top 5 rows

In [0]:
#converting to RDD
originAndDestMap = data.rdd.map(lambda x :(x["ORIGIN_AIRPORT_ID"], x["DEST_AIRPORT_ID"])).cache()
originAndDestMap.take(5)

Out[36]: [('10006', '10056'),
 ('10006', '10056'),
 ('10011', '11869'),
 ('10011', '10661'),
 ('10011', '10661')]

In [0]:
#Counting the number of outlinks for each airport in origin
originOutlinks = originAndDestMap.map(lambda x: (x[0],1)).reduceByKey(lambda x,y:x+y)
originOutlinks.take(5)

Out[37]: [('10006', 2), ('10011', 3), ('10033', 2), ('10039', 1), ('10040', 1)]

In [0]:
#getting the list of incoming links for each airport in destination
destIncomingLinks = originAndDestMap.map(lambda x: (x[1],x[0])).groupByKey()
destIncomingLinks.take(5)

Out[38]: [('10056', <pyspark.resultiterable.ResultIterable at 0x7fb433642e20>),
 ('11869', <pyspark.resultiterable.ResultIterable at 0x7fb4336429a0>),
 ('10661', <pyspark.resultiterable.ResultIterable at 0x7fb433642c10>),
 ('14709', <pyspark.resultiterable.ResultIterable at 0x7fb433642f70>),
 ('10299', <pyspark.resultiterable.ResultIterable at 0x7fb42bdc3e80>)]

In [0]:
#setting the initial default value of pageRank to 10 for all airports
pageRank = originAndDestMap.map(lambda x: x[0]).distinct().map(lambda x: (x,10))
pageRank.take(5)

Out[39]: [('10006', 10), ('10011', 10), ('10033', 10), ('10039', 10), ('10040', 10)]

In [0]:
#creating a list of airport id we are going to calculate page ranks 
originIds = originAndDestMap.map(lambda x:x[0]).distinct()
originIds.take(5)

Out[40]: ['10006', '10011', '10033', '10039', '10040']

In [0]:
#number of airports
N = originIds.count()
N

Out[41]: 840

In [0]:
#collecting the data as dictionary
originIdsList = originIds.collect()
origToDestDict =originAndDestMap.collectAsMap()
origInlinkDict = destIncomingLinks.collectAsMap()
origOutlinkDict = originOutlinks.collectAsMap()
pageRanks = pageRank.collectAsMap()

In [0]:
#applying the formula and calculating page rank
originInlinkKeys = origInlinkDict.keys()
pageRanksKeys = pageRanks.keys()
for i in range(10):
  for id in originIdsList:
    pr = 0
    if(id in originInlinkKeys):
      inlinkList = origInlinkDict[id]
      for incoming in inlinkList:
        if incoming in pageRanksKeys:
          pr+=pageRanks[incoming]/origOutlinkDict[incoming]
      pageRanks[id]=(0.15/N)+(0.85*pr)

In [0]:
#sorting the page ranks in descending order
sorted(pageRanks.items(),key=lambda x : -x[1])

Out[44]: [('11292', 23.70195962118773),
 ('11630', 23.53060325556024),
 ('13930', 23.414910538000132),
 ('11298', 20.963158514671342),
 ('10397', 20.131690557166667),
 ('13796', 15.997289078233733),
 ('10423', 15.64132131610369),
 ('12892', 15.40590411349603),
 ('12266', 15.277096665840904),
 ('12889', 14.50727693764514),
 ('14107', 14.342207407290369),
 ('11057', 13.696399548462962),
 ('13487', 13.005944729282167),
 ('13204', 12.557381285448857),
 ('10299', 12.410219330498204),
 ('14747', 12.362979532702138),
 ('11433', 12.198204087737412),
 ('13244', 12.098588746227572),
 ('11618', 11.835847194233342),
 ('14869', 11.438408363475778),
 ('11697', 10.34415172185576),
 ('10185', 10.030225758430834),
 ('10419', 10),
 ('11391', 10),
 ('11517', 10),
 ('11979', 10),
 ('12216', 10),
 ('13662', 10),
 ('14197', 10),
 ('14332', 10),
 ('14475', 10),
 ('14706', 10),
 ('14736', 10),
 ('15108', 10),
 ('15383', 10),
 ('15525', 10),
 ('15879', 10),
 ('15618', 9.880077071496498),
 ('12945', 9.762663074625545),
 ('14100', 9.687501350007041),
 ('13290', 9.265675732792502),
 ('13303', 9.220008314083685),
 ('12264', 9.019856462074783),
 ('14730', 8.991974508026946),
 ('10551', 8.756740075029244),
 ('14533', 8.744373352079736),
 ('13697', 8.529894966218006),
 ('10927', 8.500178571428572),
 ('10721', 8.307192481578062),
 ('14771', 8.212429356732057),
 ('10821', 8.088776798129942),
 ('13891', 8.046410692235847),
 ('15304', 8.042203487559325),
 ('11193', 8.022913949141),
 ('14057', 7.614776112243281),
 ('12478', 7.453328403344067),
 ('10472', 7.225330357142857),
 ('10693', 7.134400557662826),
 ('15016', 6.940176545923429),
 ('11615', 6.73551192028063),
 ('12173', 6.646399900220842),
 ('12339', 6.619497176320584),
 ('15138', 5.200544769853917),
 ('14635', 5.089641454754189),
 ('12127', 4.906740670500106),
 ('13232', 4.785944276341222),
 ('14492', 4.725914207659783),
 ('14679', 4.678345497406259),
 ('10056', 4.585933915663121),
 ('14683', 4.5387986767767705),
 ('13495', 4.417255361624745),
 ('11042', 4.177691725840697),
 ('12191', 4.139634785904751),
 ('13198', 4.124194677122405),
 ('11278', 4.047526986218604),
 ('12953', 3.866910044304115),
 ('13830', 3.8338516948125934),
 ('14831', 3.8230446882194054),
 ('14893', 3.800661623533186),
 ('15412', 3.7278887214477),
 ('10154', 3.692045691481676),
 ('14122', 3.6382329977255927),
 ('14689', 3.6308616855284837),
 ('13541', 3.585040077268976),
 ('15422', 3.5607435712579725),
 ('11259', 3.535178013149504),
 ('14027', 3.3576240704298685),
 ('12197', 3.255919736433148),
 ('14843', 3.2351029386729864),
 ('13342', 3.2168194807373403),
 ('10466', 3.178199733381766),
 ('14570', 3.0946800989666503),
 ('11066', 3.0773349216108663),
 ('15601', 3.0277272128713224),
 ('13873', 2.9616203128082725),
 ('13970', 2.909824868155724),
 ('14761', 2.8153336155659408),
 ('10529', 2.814214419613561),
 ('11540', 2.8060293354266013),
 ('10713', 2.77383001832158),
 ('14908', 2.763611896613225),
 ('11884', 2.6515087401857165),
 ('12451', 2.4417472358428953),
 ('10792', 2.4039582029074578),
 ('11844', 2.3822569799867632),
 ('14524', 2.371390056752654),
 ('11995', 2.3708613584728764),
 ('11973', 2.3644188515587667),
 ('13871', 2.337289100964337),
 ('10245', 2.3288602473642195),
 ('10140', 2.324390770400809),
 ('11833', 2.305543407302028),
 ('10994', 2.2387575671187943),
 ('14512', 2.197791391380535),
 ('11423', 2.1847684342240576),
 ('13851', 2.137251304628317),
 ('12758', 2.12018658116363),
 ('15741', 2.109783949502719),
 ('12896', 2.0898680322785044),
 ('14262', 2.058823824008913),
 ('13931', 2.0285902713264328),
 ('10304', 1.9869440189499115),
 ('14986', 1.930738730817731),
 ('10747', 1.9221786700455166),
 ('11986', 1.8965224284104423),
 ('11027', 1.8941389501963437),
 ('15581', 1.8726519825472212),
 ('16091', 1.8711049359026262),
 ('10170', 1.8492222992913363),
 ('10155', 1.845355471165987),
 ('15370', 1.826136687862492),
 ('12523', 1.7786974333646872),
 ('14307', 1.7652708682961344),
 ('10849', 1.7392572526785834),
 ('14082', 1.71966

In [0]:
#converting the page ranks to RDD and saving in output file
sc.parallelize(pageRanks).saveAsTextFile("/FileStore/tables/Output_file1.txt")